In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

# pd.set_option('max_colwidth', 100)


In [33]:
mf = pd.read_csv('buildstock_MF.csv', index_col='Building')
sfa = pd.read_csv('buildstock_SFA.csv', index_col='Building')

In [34]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [41]:
int(df.loc[1]['Geometry Building Number Units MF'])

54

In [43]:
master_df = pd.DataFrame()
buildtype = 'MF'
df = mf
middle_ct = 0
for idx in df.index:
    bldg = df.loc[idx]
    units = int(bldg[f'Geometry Building Number Units {buildtype}'])
#     bstock = pd.DataFrame(columns= df.columns, index=range(int(1),units+1))
    bstock = pd.DataFrame(columns= df.columns)
    bstock.index.name = 'Building'
    
    if units >= 3 and buildtype == 'SFA':
        mid_mult = units-2
        units = 3
    else:
        mid_mult = np.NaN
#     print("ID: ", idx)
#     print("floors: ", floors)
#     print("units: ", units)   
    
    if buildtype == 'MF':
        not_type = 'SFA'
        floors = int(bldg['Geometry Stories'])
        num_units_per_floor = units/floors
        front_orient = bldg['Orientation']
        rear_orient = front_orient
        num_middle_z = floors - 2
        if num_units_per_floor%2 != 0: #no rear
            has_rear_units = False
            num_middle_x = int(num_units_per_floor - 2)
        else: #rear units
            has_rear_units = True
            rear_orient = orient_dict[bldg['Orientation']]
            num_middle_x = int((num_units_per_floor/2) - 2)   
        if num_middle_x < 0:
            num_middle_x = 0
        if has_rear_units:
            num_units_front = int(units/2)
            units_add = int(num_units_per_floor/2)
        else:
            num_units_front = int(units)
            units_add = int(num_units_per_floor)
    else:
        not_type = 'MF'
        num_middle_x = units - 2
        has_rear_units = False

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if (buildtype=='SFA') or (num_units_front/floors > 1 and buildtype=='MF'):
        h_right = [num_middle_x + 2]
    else:
        h_right = []
        
    #Define horizontal locatons by floor
    if buildtype == 'MF':
        for f in range(2,floors+1):
            h_left += [h_left[-1] + units_add]
            h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
            h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    if buildtype == 'MF':
        #..bottom
        if num_units_front/floors > 1:
            lv_bottom = list(range(1,2+num_middle_x+1))
        else:
            lv_bottom = [1]
        #..middle
        if num_middle_z > 0:
            lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
        else:
            lv_middle = []
        #..top
        if floors > 1:
            lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
        else:
            lv_top = []

        if has_rear_units:
            h_left += list(map(lambda x:x+num_units_front, h_left))
            h_mid += list(map(lambda x:x+num_units_front, h_mid))
            h_right += list(map(lambda x:x+num_units_front, h_right))
            lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
            lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
            lv_top += list(map(lambda x:x+num_units_front, lv_top))

    floor_unit = 1
   
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg
        bstock.loc[unit, f'Geometry Building Horizontal Location {not_type}'] = 'N/A'
        if unit in h_left:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Left'
        elif unit in h_mid:
            middle_ct += 1
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Right'

        if buildtype == 'MF':
            if unit in lv_bottom:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Bottom'
            elif unit in lv_middle:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Middle'
            elif unit in lv_top:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Top'

            if unit <= units/2:
                bstock.loc[unit, 'Orientation'] = front_orient
            else:
                bstock.loc[unit, 'Orientation'] = rear_orient
        else:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'N/A'
            
#         bstock.loc[unit, 'Geometry Single Unit Has Rear Units'] = has_rear_units
    bstock['building_code'] = 'bldg_{}'.format(idx)
    bstock['middle_multiplier'] = mid_mult
    
    if master_df.empty:
        master_df = bstock.copy()
    else:
        master_df = master_df.append(bstock)
#     bstock.to_csv('./mf_buildstock_singles/buildstock_' + str(idx) + '.csv')

    
master_df.index= range(1,len(master_df)+1)
master_df.index.rename('Building', inplace=True)

In [44]:
bldg_map = pd.DataFrame(columns=['building_range', 'mid_multiplier'], index=master_df.building_code.unique())
for bldg in master_df.building_code.unique():
    idx = list(master_df[master_df.building_code == bldg].index.values)
    bldg_map.loc[bldg]['building_range'] = '"{}"'.format(str(idx).replace(' ','').strip(']['))
    bldg_map.loc[bldg]['mid_multiplier'] = master_df[master_df.building_code == bldg].iloc[0].middle_multiplier
bldg_map.head()
bldg_map.to_csv(f'./building_id_mapping_{buildtype}.csv')

In [45]:
master_df = master_df.drop(['building_code', 'middle_multiplier'], axis=1)
master_df.to_csv(f'buildstock_{buildtype}_single.csv')

In [47]:
len(master_df)

6080

In [41]:
bstock.filter(['Geometry Building Level MF', 'Geometry Building Horizontal Location MF', 'Orientation', 'Geometry Stories', 'Geometry Building Number Units MF'])

,Geometry Building Level MF,Geometry Building Horizontal Location MF,Orientation,Geometry Stories,Geometry Building Number Units MF
Building,,,,,
1,Bottom,Left,North,1,4
2,Bottom,Right,North,1,4
3,Bottom,Left,South,1,4
4,Bottom,Right,South,1,4
